In [1]:
import pandas as pd
import numpy as np
import glob
import sys
import re
from astropy.cosmology import Planck15 as cosmo
import rate_functions as functions   ### Mike's code                                                                                                                                
import astropy.units as u
import matplotlib.pyplot as plt
import scipy

In [2]:
## calculate the mass fraction of mergers for this merger redshift bin
def find_f_z(pop, zmbin_low, zmbin_high, zf_mid, this_Z, mets, M_tot):
    
    Zsun = 0.017
    sigmaZ = 0.5
    lowZ = Zsun/200
    highZ = 2*Zsun
    
    # assuming all systems are born in this formation bin, find how many merge in this merger bin                                                                                  
    if zmbin_low==0:
        # special treatment for the first merger bin                                                                                                                               
        tdelay_max = 0
        tdelay_min = cosmo.lookback_time(zmbin_high).to(u.Myr).value

    else:
        tdelay_max = cosmo.lookback_time(zf_mid).to(u.Myr).value - cosmo.lookback_time(zmbin_low).to(u.Myr).value
        tdelay_min = cosmo.lookback_time(zf_mid).to(u.Myr).value - cosmo.lookback_time(zmbin_high).to(u.Myr).value
        
        #print (tdelay_min, tdelay_max)

    N_merge_CP = len(pop.loc[(pop['tphys']<=tdelay_max) & (pop['tphys']>tdelay_min)])
        
        
    # the relative weight of this metallicity at this particular redshift                                    
    this_Z_prob = functions.metal_disp_z(zf_mid, this_Z, sigmaZ, lowZ, highZ)
    #print (this_Z_prob)
                    
    ### sum of conditional probabilities: P(Z|zf_mid)                                                                            
    all_Z_prob_sum = np.sum(functions.metal_disp_z(np.ones(len(mets))*zf_mid, mets, sigmaZ, lowZ, highZ))
    N_merge = N_merge_CP*this_Z_prob/all_Z_prob_sum

    f_zm = float(N_merge)/M_tot
        
    return f_zm


In [3]:
## calculate the volumetric rate for the given population
## returns array of volumetric rates across mass bins
def volumetric_rates(population, f_loc_arr, this_Z, M_tot, mets, zfbins, zmbins):
    
    f_loc_Z = []
    merge_rates = []  ## volumetric merger rates
    
    ## constants
    prefactor = 1 / cosmo.H(0).to(u.Gpc * u.Gpc**-1 * u.yr**-1).value
    
    ## redshift range for local universe
    zmbin_low = np.min(zmbins)
    zmbin_high = np.max(zmbins)
     
    i = 0
    for zfbin_low, zfbin_high in zip(zfbins[::-1][1:], zfbins[::-1][:-1]):      
                                          
        ### calculate redshift/metallicity weighting                                                                                                                                   
        # get redshift in the middle of this log-spaced formation redshift interval                                                                                                    
        zf_mid = 10**(np.log10(zfbin_low) + (np.log10(zfbin_high)-np.log10(zfbin_low))/2.0)
        zm_mid = 10**(np.log10(zmbin_low) + (np.log10(zmbin_high)-np.log10(zmbin_low))/2.0)
            
        ## option for single metallicity populations
        if f_loc_arr is None:        
            f_zm = find_f_z(population, zmbin_low, zmbin_high, zf_mid, this_Z, mets, M_tot)
            f_loc_Z.append(f_zm)
            
        ## option for full convolved metallicity population
        else:  
            f_zm = np.sum(f_loc_arr[:, i])
            #print (f_zm)
            f_loc_Z = None 
            i += 1
                    
        # cosmological factor                                                                                                                                                          
        E_zf = (cosmo._Onu0*(1+zf_mid)**4 + cosmo._Om0*(1+zf_mid)**3 + cosmo._Ok0*(1+zf_mid)**2 + \
                cosmo._Ode0)**(1./2)

                    
        SFR = functions.sfr_z(zf_mid, mdl='2017')*(u.Mpc**-3)   ## SFR in Msun / Mpc^3 / yr
        t_loc = cosmo.lookback_time(zmbin_high).to(u.yr).value
                    
        ## merger rate for this [zf, zm] in Gpc^-3 yr^-1
        this_merge_Rate =  prefactor * 1/t_loc * f_zm * SFR.to(u.Gpc**-3).value/ ((1+zf_mid)*E_zf) \
                            * (zfbin_high - zfbin_low)
                    
        #print (this_merge_Rate)         
        merge_rates.append(this_merge_Rate)

    
    return merge_rates, f_loc_Z


In [4]:
### path to folder with COSMIC metallicity runs                                                                                                                                               
COSMIC_path = "pessimistic_CE_runs/*"
all_COSMIC_runs = glob.glob(COSMIC_path)

In [5]:
Zsun = 0.017    ## Solar metallicity                                                                                                                                                         
                                                                                                                                                                                                                                                          
N_zbins = 1000   ## number of redshift bins                                                                                                                                                   
zmin = 0         ## lower bound of redshift space                                                                                                                                             
zmax_f = 15        ## upper bound of formation redshift space 
zmax_m = 0.1    ## upper bound of merger redshift space

In [6]:
### array of metallicities, should match those of COSMIC_runs                                                                                                                                 
metallicities = [Zsun/200, Zsun/100, Zsun/20, Zsun/10, Zsun/5, Zsun/2, Zsun]

### create log-spaced redshift bins                                                                                                                                                           
if zmin==0:
    # account for log-spaced bins                                                                                                                                                             
    zmin = 1e-3
zfbins = np.logspace(np.log10(zmin), np.log10(zmax_f), N_zbins+1)
zmbins = np.logspace(np.log10(zmin), np.log10(zmax_m), N_zbins+1)

In [7]:
## loop through all COSMIC populations and calculate BBHm merger rates
all_BBH_pop_mbin_rates = []
all_tot_pop_mbin_rates = []
all_BBH_floc_arr = []
all_tot_floc_arr = []
read_mets = []

for pop_dir in all_COSMIC_runs:
    
    print ("population: ", pop_dir)
    
    data = glob.glob(pop_dir + "/*.h5")[0]
    this_Z_frac = float(re.findall('\d+.\d+', data)[0]) ## parse population metallicity from filename
    this_Z = round(this_Z_frac*Zsun, 6)
    read_mets.append(this_Z_frac)
                        
    ### get total sampled mass of this population                                                                                                                                             
    Mtot_arr = pd.read_hdf(data, key='mass_stars') 
    Mtot = Mtot_arr.iloc[-1].values[0]
    
    bpp =  pd.read_hdf(data, key='bpp')
    bcm = pd.read_hdf(data, key='bcm')
    
    ## find BBH mergers using bcm array                                                                                                                             
    BBHm_bcm = bcm.loc[bcm["merger_type"] == "1414"]
    BBHm_bin = BBHm_bcm['bin_num'].values
    
    #print ("Number of CP BBH mergers: ", len(BBHm_bin))
    BBHm_bpp = bpp.loc[BBHm_bin]

    all_mergers = bpp.loc[(bpp['evol_type']==6)]
    BBH_mergers = BBHm_bpp.loc[BBHm_bpp["evol_type"]==6]
    
    
    ## detection rates for this metallicity population, separated by mass bin
    BBH_rate_array, f_loc_BBH_array = volumetric_rates(BBH_mergers, None, this_Z, Mtot, metallicities, zfbins, zmbins)
    tot_rate_array, f_loc_tot_array = volumetric_rates(all_mergers, None, this_Z, Mtot, metallicities, zfbins, zmbins)
    
    print ("local BBH rate: ", np.sum(BBH_rate_array))
    print ("local DCO rate: ", np.sum(tot_rate_array))
    
    
    all_BBH_floc_arr.append(f_loc_BBH_array)
    all_tot_floc_arr.append(f_loc_tot_array)
   
    BBH_rate_array = []
    tot_rate_array = []
    f_loc_BBH_array = []
    f_loc_tot_array = []
    

population:  pessimistic_CE_runs/0.05_Zsun
local BBH rate:  22.622173490080474
local DCO rate:  30.828481308540177
population:  pessimistic_CE_runs/0.01_Zsun
local BBH rate:  0.9061027686833513
local DCO rate:  0.9956996406381149
population:  pessimistic_CE_runs/1.0_Zsun
local BBH rate:  1.1956718862985103
local DCO rate:  198.31888509179873
population:  pessimistic_CE_runs/0.1_Zsun
local BBH rate:  50.56608613283651
local DCO rate:  73.04798937000402
population:  pessimistic_CE_runs/0.005_Zsun
local BBH rate:  0.4458118803020819
local DCO rate:  0.47667700307413263
population:  pessimistic_CE_runs/0.2_Zsun
local BBH rate:  23.70734018932558
local DCO rate:  107.74078845832695
population:  pessimistic_CE_runs/0.5_Zsun
local BBH rate:  1.4541764612581838
local DCO rate:  214.77078113661275


In [31]:
### using f_loc arrays from above, calculate the total local volumetric rate for all metallicities

all_BBH_floc_arr = np.array(all_BBH_floc_arr)
all_tot_floc_arr = np.array(all_tot_floc_arr)

BBH_rate_array, empty  = volumetric_rates(BBH_mergers, all_BBH_floc_arr, this_Z, Mtot, metallicities, zfbins, zmbins)
tot_rate_array, empty  = volumetric_rates(all_mergers, all_tot_floc_arr, this_Z, Mtot, metallicities, zfbins, zmbins)

print ("total convolved BBH rate: ", np.sum(BBH_rate_array))
print ("total convolved DCO rate: ", np.sum(tot_rate_array))

total convolved BBH rate:  100.89736280878469
total convolved DCO rate:  626.1793020089949
